# Azure OpenAI to identify PII within data

## Few-shot learning

The OpenAI completion API provides an endpoint where the prompt and additional parameter can be posted to. Within the prompt [few-shot learning](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/advanced-prompt-engineering?pivots=programming-language-chat-completions) is used to provide a set of training examples as part of the prompt to give context to the model.
```md
Example 1:
Input: Susanne is a power user and works daily 8 hours with the system
Response: xx is a power user and works daily 8 hours with the system

Example 2:
Input: Charles has used Outlook 15 minutes yesterday
Response: xx has used Outlook 15 minutes yesterday

Example 3:
Input: Robert is driving a car at high speed
Response: xx is driving a car at high speed

Example 4:
Input: Jon has an invitation to a restaurant
Response: xx has an invitation to a restaurant

Example 5:
Input: Olivia is running fast during the competition
Response: xx is running fast during the competition
```

## Parameter:

- ***Temperature:*** Set to 0.7 to produce more repetitive and deterministic responses
- ***Top P:*** Set to 0.95 to narrow the model'S token selection to likelier tokens
- ***Stop:*** Set to Newline to stop the to stop the model generating further tokens after the first response
- ***Presence,Frequency Penalty:*** Set to 0 to limit the chance of repeating tokens

In [19]:
using System.Net; 
using System.Net.Http; 
using System.Text.Json; 

var payload = new { 
    prompt = "You are a tool which extracts personal identifiable information (PII) from data provided to you. \nIf you are unsure if there\u0027s PII in the data answer with \u0022The provided dataset needs checking\u0022.\nDo not invent additional information.\nDo not include potential PII information in questions you add to your response.\nExample 1:\nInput: Susanne is a power user and works daily 8 hours with the system\nResponse: xx is a power user and works daily 8 hours with the system\n\nExample 2:\nInput: Charles has used Outlook 15 minutes yesterday\nResponse: xx has used Outlook 15 minutes yesterday\n\nExample 3:\nInput: Robert is driving a car at high speed\nResponse: xx is driving a car at high speed\n\nExample 4:\nInput: Jon has an invitation to a restaurant\nResponse: xx has an invitation to a restaurant\n\nExample 5:\nInput: Olivia is running fast during the competition\nResponse: xx is running fast during the competition\n\n'Ashley has bought a news paper at the airport'",
    max_tokens = 150,
    temperature = (float)0.7,
    stop="\n",
    presence_penalty=(float)0,
    frequency_penalty=(float)0 
};
string jsonPayload = JsonSerializer.Serialize(payload);

#!share --from c# jsonPayload --as jsonPayload 

## Endpoint, API-Key & deployment name definition

Replace endPoint, apiKey, deploymentName with values from your deployment. If you used [the containing script](/src/CreateEnv/CreateEnv.azcli) to create the environment the necessary information can be copied from:  

- `$csEndpoint`
- `$csApiKey`
- `$modelDEploymentName`

Sample values (not working / replace with your values): 

```csharp
    string endPoint = "https://southcentralus.api.cognitive.microsoft.com/";
    string apiKey = "5a5477c4cdea4999ab0ca675259e3530";
    string deploymentName = "gpt35turbo-deployment";
```

In [22]:
string endPoint = "<<INSERT your Azure OpenAI endpoint>>";
string apiKey = "<<INSERT your Azure OpenAI API key>>";
string deploymentName = "<<INSERT your Azure OpenAI deployment name";
string url = $"{endPoint}openai/deployments/{deploymentName}/completions?api-version=2023-03-15-preview";

#!share --from c# url --as url 
#!share --from c# apiKey --as apiKey
#!share --from c# deploymentName --as deploymentName

## Invocation model API

.net HttpClient is used to invoke the model API.

Please see the response from the model: ***xx has bought a news paper at the airport*** as completion for 'Ashley has bought a news paper at the airport'


In [25]:
HttpRequestMessage httpRequestMessage = new HttpRequestMessage(HttpMethod.Post, url);
httpRequestMessage.Headers.TryAddWithoutValidation("Content-Type", "application/json");
httpRequestMessage.Headers.Add("api-key", apiKey); 
httpRequestMessage.Content = new StringContent(jsonPayload, System.Text.Encoding.UTF8, "application/json");

using (HttpClient httpClient = new HttpClient()) {
    HttpResponseMessage httpResponseMessage = await httpClient.SendAsync(httpRequestMessage);
    string response = await httpResponseMessage.Content.ReadAsStringAsync();
    Console.WriteLine(response);
}

{"id":"cmpl-7Ie7EoU8RoQb4PyTxKz9CaVyRa4j4","object":"text_completion","created":1684678464,"model":"gpt-35-turbo","choices":[{"text":" =\u003e 'xx has bought a news paper at the airport'","index":0,"finish_reason":"stop","logprobs":null}],"usage":{"completion_tokens":12,"prompt_tokens":213,"total_tokens":225}}

